In [76]:
import pandas as pd
import json
from generate_graph import get_propositions, generateEdges, createGraph, get_propositions_nosplit
from refine_graph import refine
from query_graph import QueryGraph
from tqdm import tqdm
tqdm.pandas()

## Dataset

In [78]:
from PyPDF2 import PdfReader
# provide the path of  pdf file/files.
pdfreader = PdfReader('datasets/facultymanual/facultymanual.pdf')

In [79]:
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [80]:
import tiktoken

# Use GPT-4o encoding
enc = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = enc.encode(raw_text)
print(f"Total tokens: {len(raw_text)}")

Total tokens: 347451


In [83]:
# Title:  
# FACULTY  MANUAL  Reference No.  OPR – HRD – D – M – 003 
# DEPARTMENT  HUMAN RESOURCES  
# This is a controlled document. Photocopying or printing of this document, without the controlled mark, shall make this document an 
# uncontrolled copy.  
 
# Date of Effectivity  1 August  2023 Control Mark  
# QMO Reviewed  Page 1 of 147 
# Supersedes  1 December 2019  

# Define lists of known header/footer phrases
common_headers = ["Title:", "FACULTY  MANUAL  Reference No.", "DEPARTMENT  HUMAN RESOURCES", "This is a controlled document.", "uncontrolled copy."]
common_footers = ["Date of Effectivity", "QMO Reviewed", "Supersedes"]

page_texts = []

for page in pdfreader.pages:
    text = page.extract_text() or ""
    lines = text.splitlines()

    # Filter out lines that contain any header/footer keywords
    filtered = [
        line for line in lines
        if not any(keyword in line for keyword in common_headers + common_footers)
    ]

    page_texts.append("\n".join(filtered))

In [86]:
page_texts[147:148]
# len(page_texts)

[' \n 2) Microsoft certification, including being named a Microsoft Innovative Education \nExpert (MIEE), can be credited under Professional Certification.  \nCategory  Points  \nConsultancy  5 \nImmersion  5 \nFellowship  5 \nFaculty Exchange  5 \nProfessional Certification  5 \nNational Government Appointment  5 \nd. Recognition and Awards (Maximum of 5 Points)  \nCategory  Points  \nInternational  5 \nNational  3 \nInstitutional  2 \n \ne. Professional Organization (Maximum of 5 Points)  \n1) The faculty should be a member of the professional organization within the \nevaluation period.  \n2) Advanced Member refers to all other membership types that undergo screening.  \nCategory  Points  \nOfficer  5 \nCommittee Member  3 \nAdvanced Member  3 \nRegular Member  1 \n \n \nVII. FORMS  \nA. SPES Form  \nB. SPES Colle ge Summary  \nC. Classroom Observation  \n \nVIII. EFFECTIVITY  \nThis Revised SPES for Faculty is effective Academic Year 2023 -2024.  \n \n ']

## Experiments

### Initial Experiment

In [ ]:
# propositions = []

# for context in enumerate(tqdm(mylist)):
#     get_propositions(context, propositions)

In [88]:
propositions = []

for context in enumerate(tqdm(page_texts)):
    get_propositions_nosplit(context, propositions)

100%|██████████| 148/148 [44:22<00:00, 17.99s/it]


took 44m and 22s

In [89]:
# Open the file in read mode
with open('propositions_facultymanual.txt', 'r') as file:
    # Read all lines and store them in a list
    propositions_from_file = [line.strip() for line in file]

print(propositions_from_file)

['Education that works.', 'FACULTY MANUAL', 'The manual version control log contains various versions of the faculty manual.', 'The first version was consolidated in 2015.', 'The 2015 version consolidated policies and procedures specific to faculty for the academic years 2014-2015 to 2015-2016.', 'The 2015 version was submitted to TUV Rheinland during the 2016 ISO Audit.', 'The second version was revised in 2017.', 'The 2017 version revised the 2015 faculty manual due to changes in the organizational structure and other employee policies and procedures.', 'The 2017 version aligned with NU’s 5-year development plan for the academic years 2016-2017 to 2017-2018.', 'The 2017 version was submitted to TUV Rheinland during the 2017 ISO Audit.', 'The document identified as ACO–AO–D–M–001 involved a revision of the policies and procedures due to changes in the organization structure.', 'The revision considered the ongoing university expansion.', 'The ACO–AO–D–M–001 revision was submitted to TU

In [90]:
len(propositions_from_file)

3693

In [91]:
list_of_edges = generateEdges(propositions_from_file)

100%|██████████| 3693/3693 [2:56:13<00:00,  2.86s/it]  


took 176m and 13.1s

In [92]:
if createGraph(list_of_edges):
    print("Success")
else:
    print("Failed")

Success
